Scope:
- Single year: 2024
- AIS coverage: non-EU waters only
- Assumption: CO₂ intensity per nautical mile is vessel- and operation-specific,
  not region-specific, for comparable operating conditions
- Intended use: screening & verification, not final compliance determination

In [1]:
import polars as pl
from pathlib import Path
import math

# ============================
# CONFIG
# ============================

DATA_ROOT = Path("/Users/jakobschneider/Machine Learning/Data_LCC")

# Your cleaned, enriched AIS logs (no MMSI, already mapped to IMO)
AIS_ENRICHED_PATH = DATA_ROOT / "AIS_2024_enriched_optimized.parquet"

# Optional: ship metadata (only if you want Length/Width/VesselType/Draft features)
AIS_SHIPS_PATH = DATA_ROOT / "AIS_2024_enriched_optimized.parquet"

# MRV file for targets
MRV_PATH = DATA_ROOT / "MRV_2024.xlsx"

YEAR = 2024

# Segment filters / thresholds (documented assumptions)
MAX_DT_SECONDS = 2 * 3600     # max 2h gap between AIS points
MAX_SPEED_KN = 35             # teleport filter based on implied speed

# Data quality thresholds (tunable)
MIN_DISTANCE_NM = 500
MIN_POINTS = 10_000
MAX_MEDIAN_DT = 600           # 10 minutes


# ============================
# HELPERS
# ============================

def haversine_nm(lat1: pl.Expr, lon1: pl.Expr, lat2: pl.Expr, lon2: pl.Expr) -> pl.Expr:
    """
    Haversine distance in nautical miles (nm), fully vectorized with Polars Expr methods.
    Inputs must be Polars expressions in degrees.
    """
    R_km = 6371.0
    deg2rad = math.pi / 180.0

    lat1r = lat1 * deg2rad
    lon1r = lon1 * deg2rad
    lat2r = lat2 * deg2rad
    lon2r = lon2 * deg2rad

    dlat = lat2r - lat1r
    dlon = lon2r - lon1r

    a = (dlat / 2).sin() ** 2 + lat1r.cos() * lat2r.cos() * (dlon / 2).sin() ** 2
    c = (a.sqrt()).arcsin() * 2.0

    km = c * R_km
    nm = km / 1.852
    return nm


# ============================
# STEP 1: Load AIS logs (Lazy, memory-safe)
# ============================

lf_ais = (
    pl.scan_parquet(AIS_ENRICHED_PATH)
    .select([
        "IMO",          # <-- key column (since MMSI is not present anymore)
        "BaseDateTime",
        "LAT",
        "LON",
        "SOG",
    ])
    # Keep only plausible raw rows
    .filter(
        pl.col("IMO").is_not_null() &
        pl.col("BaseDateTime").is_not_null() &
        pl.col("LAT").is_not_null() &
        pl.col("LON").is_not_null() &
        (pl.col("SOG") >= 0)
    )
)

# ============================
# STEP 2: Compute dt per IMO and filter large gaps
# ============================

lf_ais = (
    lf_ais
    .sort(["IMO", "BaseDateTime"])
    .with_columns(
        dt_seconds=(
            pl.col("BaseDateTime")
            .diff()
            .over("IMO")
            .dt.total_seconds()
        )
    )
    # Drop first row per group (dt null) + invalid / huge gaps
    .filter(
        pl.col("dt_seconds").is_not_null() &
        (pl.col("dt_seconds") > 0) &
        (pl.col("dt_seconds") < MAX_DT_SECONDS)
    )
)

# ============================
# STEP 3: Segment distance (nm) + teleport filter
# ============================

lf_ais = (
    lf_ais
    .with_columns(
        segment_distance_nm=haversine_nm(
            pl.col("LAT").shift(1).over("IMO"),
            pl.col("LON").shift(1).over("IMO"),
            pl.col("LAT"),
            pl.col("LON"),
        )
    )
    .with_columns(
        implied_speed_kn=pl.col("segment_distance_nm") / (pl.col("dt_seconds") / 3600.0)
    )
)

# ============================
# STEP 4: Movement proxies (simple & robust)
# ============================

lf_ais = lf_ais.with_columns([
    (pl.col("SOG") > 1.0).cast(pl.Int8).alias("moving_flag"),
    (pl.col("SOG") < 0.5).cast(pl.Int8).alias("idle_flag"),
])

# ============================
# STEP 5: Aggregate to IMO × 2024
# One row per ship (IMO) for the full year 2024
# ============================

lf_imo_agg = (
    lf_ais
    .group_by("IMO")
    .agg([
        pl.sum("segment_distance_nm").alias("ais_distance_nm_total"),
        (pl.sum("dt_seconds") / 3600.0).alias("ais_time_hours_total"),
        pl.count().alias("ais_points"),

        pl.mean("SOG").alias("sog_mean_kn"),
        pl.quantile("SOG", 0.50).alias("sog_p50_kn"),
        pl.quantile("SOG", 0.95).alias("sog_p95_kn"),

        ((pl.col("dt_seconds") * pl.col("moving_flag")).sum() / 3600.0).alias("moving_hours"),
        ((pl.col("dt_seconds") * pl.col("idle_flag")).sum() / 3600.0).alias("idle_hours"),

        pl.median("dt_seconds").alias("median_dt_seconds"),
    ])
    .with_columns([
        pl.lit(YEAR).alias("year"),
        (pl.col("moving_hours") / pl.col("ais_time_hours_total")).alias("moving_share"),
    ])
)

# ============================
# STEP 6 (Optional): Add ship metadata features
# Only do this if AIS_SHIPS_PATH exists and has reliable IMO mapping
# ============================

ADD_SHIP_METADATA = True

if ADD_SHIP_METADATA:
    df_ships = (
        pl.read_parquet(AIS_SHIPS_PATH)
        .select([
            "IMO",
            "VesselType",
            "Length",
            "Width",
            "Draft",
        ])
        .filter(pl.col("IMO").is_not_null())
        # One row per IMO (choose robust aggregations)
        .group_by("IMO")
        .agg([
            pl.first("VesselType").alias("VesselType"),
            pl.median("Length").alias("Length"),
            pl.median("Width").alias("Width"),
            pl.median("Draft").alias("draft_m_median"),
        ])
    )

    lf_imo_agg = lf_imo_agg.join(df_ships, on="IMO", how="left")

# ============================
# STEP 7: Quality flags (for training / verification gating)
# ============================

lf_imo_agg = lf_imo_agg.with_columns(
    quality_ok=(
        (pl.col("ais_distance_nm_total") > MIN_DISTANCE_NM) &
        (pl.col("ais_points") > MIN_POINTS) &
        (pl.col("median_dt_seconds") < MAX_MEDIAN_DT)
    )
)

# ============================
# STEP 8: Load MRV target (2024 only) and join
# ============================

df_mrv = (
    pl.read_excel(MRV_PATH)
    .select([
        pl.col("IMO Number").alias("IMO"),
        pl.col("Reporting Period").alias("year"),
        pl.col("CO₂ emissions per distance [kg CO₂ / n mile]").alias("y_co2_per_nm_kg"),
    ])
    .filter(pl.col("year") == YEAR)
)

df_final = (
    lf_imo_agg
    .join(df_mrv, on=["IMO", "year"], how="inner")
    .collect()
)

# ============================
# STEP 9: Final model table (inspect / save)
# ============================

df_model = df_final.select([
    "IMO",
    "year",
    "y_co2_per_nm_kg",

    "ais_distance_nm_total",
    "ais_time_hours_total",
    "moving_share",

    "sog_mean_kn",
    "sog_p50_kn",
    "sog_p95_kn",

    # Optional columns (present only if ADD_SHIP_METADATA=True)
    *[c for c in ["Length", "Width", "draft_m_median", "VesselType"] if c in df_final.columns],

    "quality_ok",
    "ais_points",
    "median_dt_seconds",
])

print(df_model.shape)
print(df_model.head(5))

/var/folders/s6/68z2fm6j28jfpyn0vqv3j9fw0000gn/T/ipykernel_48063/2834019136.py:145: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("ais_points"),


: 